In [116]:
import clingo

from starlingo.Atom import Atom
from starlingo.Literal import BasicLiteral
from starlingo.Program import Program, from_string, evaluate_forwards
from starlingo.Rule import NormalRule, Constraint
from starlingo.Symbol import Function, Term

In [117]:

domain_raw_str = """

effect(at(R,P2,__t+1), move(R,P2,__t), (at(R,P1,__t), adjacent(P1, P2)), (-at(_,P2,__t),)) :- robot(R), loc(P1), loc(P2).
effect(at(R,P,__t+1), -move(R,_,__t), (at(R,P,__t),), ()) :- robot(R), loc(P).

"""
domain_raw_p = from_string(domain_raw_str)[0]

In [118]:
__t = Function('__t')

In [119]:
domain_rules = []
for rule in domain_raw_p.rules:
    if rule.is_rule() and rule.is_normal_rule():
        head_symbol = rule.head.atom.symbol
        if head_symbol.match('effect', 4):
            effect_symbol = head_symbol.arguments[0]
            action_symbol = head_symbol.arguments[1]
            condition_symbols = head_symbol.arguments[2].arguments
            precondition_symbols = head_symbol.arguments[3].arguments

            effect = BasicLiteral(Atom(effect_symbol))
            action = BasicLiteral(Atom(action_symbol)).strong_neg_to_default_neg()
            conditions = tuple(BasicLiteral(Atom(condition_symbol)).strong_neg_to_default_neg() for condition_symbol in
                               condition_symbols)
            preconditions = tuple(
                -BasicLiteral(Atom(precondition_symbol)).strong_neg_to_default_neg() for precondition_symbol in
                precondition_symbols)

            effect_rule = NormalRule(effect, (*rule.body, action, *conditions))
            domain_rules.append(effect_rule)
            for precondition in preconditions:
                precondition_rule = Constraint((*rule.body, action, *conditions, precondition))
                domain_rules.append(precondition_rule)

domain_p = Program(name='domain', parameters=(__t,), rules=domain_rules)
print(domain_p.custom_str(sep='\n'))

#program domain(__t).
at(R,P2,__t+1) :- robot(R), loc(P1), loc(P2), move(R,P2,__t), at(R,P1,__t), adjacent(P1,P2).
:- robot(R), loc(P1), loc(P2), move(R,P2,__t), at(R,P1,__t), adjacent(P1,P2), at(_,P2,__t).
at(R,P,__t+1) :- robot(R), loc(P), not move(R,_,__t), at(R,P,__t).


In [120]:
domain_str = """

#program domain.

loc((X,Y)) :- X=1..3,Y=1..3, (X,Y) != (1,3), (X,Y) != (3,1).
adjacent(P1,P2) :- loc(P1), P1=(X,Y), loc(P2), P2=(X,Y+1).
adjacent(P1,P2) :- loc(P1), P1=(X,Y), loc(P2), P2=(X+1,Y).
adjacent(P1,P2) :- loc(P1), P1=(X,Y), loc(P2), P2=(X,Y-1).
adjacent(P1,P2) :- loc(P1), P1=(X,Y), loc(P2), P2=(X-1,Y).

robot(worker).
robot(charger).

1 { at(R,P,0) : loc(P) } 1 :- robot(R).

#program domain(__t).

0 { move(R,P,__t) : loc(P) } 1 :- robot(R).

"""
print(domain_str)



#program domain.

loc((X,Y)) :- X=1..3,Y=1..3, (X,Y) != (1,3), (X,Y) != (3,1).
adjacent(P1,P2) :- loc(P1), P1=(X,Y), loc(P2), P2=(X,Y+1).
adjacent(P1,P2) :- loc(P1), P1=(X,Y), loc(P2), P2=(X+1,Y).
adjacent(P1,P2) :- loc(P1), P1=(X,Y), loc(P2), P2=(X,Y-1).
adjacent(P1,P2) :- loc(P1), P1=(X,Y), loc(P2), P2=(X-1,Y).

robot(worker).
robot(charger).

1 { at(R,P,0) : loc(P) } 1 :- robot(R).

#program domain(__t).

0 { move(R,P,__t) : loc(P) } 1 :- robot(R).




In [121]:
domain_integrity_str = """

#program domain_integrity(__t).

:- robot(R), { at(R,P,__t) : loc(P) } 0.
:- robot(R), 2 { at(R,P,__t) : loc(P) }.
:- loc(P), 2 { at(R,P,__t) : robot(R) }.

"""
print(domain_integrity_str)



#program domain_integrity(__t).

:- robot(R), { at(R,P,__t) : loc(P) } 0.
:- robot(R), 2 { at(R,P,__t) : loc(P) }.
:- loc(P), 2 { at(R,P,__t) : robot(R) }.




In [122]:
init_raw_str = """

init(at(worker, (1,1))).
init(at(charger, (2,2))).

goal(at(worker, (3,3))).
goal(at(charger, (2,2))).

"""
init_raw_p = from_string(init_raw_str)[0]

In [123]:
planning_problem_rules = []
for rule in init_raw_p.rules:
    if rule.is_rule() and rule.is_normal_rule():
        head_symbol = rule.head.atom.symbol
        if head_symbol.match('init', 1):
            effect_symbol = head_symbol.arguments[0].arguments_append(Term.zero())
            effect = BasicLiteral(Atom(effect_symbol)).strong_neg_to_default_neg()

            effect_constraint = Constraint((-effect,))
            planning_problem_rules.append(effect_constraint)
        elif head_symbol.match('goal', 1):
            effect_symbol = head_symbol.arguments[0].arguments_append(__t)
            effect = BasicLiteral(Atom(effect_symbol)).strong_neg_to_default_neg()

            effect_constraint = Constraint((-effect,))
            planning_problem_rules.append(effect_constraint)

planning_problem_p = Program('planning_problem', parameters=(__t,), rules=planning_problem_rules)
print(planning_problem_p.custom_str(sep='\n'))

#program planning_problem(__t).
:- not at(worker,(1,1),0).
:- not at(charger,(2,2),0).
:- not at(worker,(3,3),__t).
:- not at(charger,(2,2),__t).


In [124]:
planning_problem_ps = (domain_p, domain_str, domain_integrity_str, planning_problem_p)
for p in planning_problem_ps:
    if isinstance(p, str):
        print(p)
    elif isinstance(p, Program):
        print(p.custom_str('\n'))
    print()

#program domain(__t).
at(R,P2,__t+1) :- robot(R), loc(P1), loc(P2), move(R,P2,__t), at(R,P1,__t), adjacent(P1,P2).
:- robot(R), loc(P1), loc(P2), move(R,P2,__t), at(R,P1,__t), adjacent(P1,P2), at(_,P2,__t).
at(R,P,__t+1) :- robot(R), loc(P), not move(R,_,__t), at(R,P,__t).



#program domain.

loc((X,Y)) :- X=1..3,Y=1..3, (X,Y) != (1,3), (X,Y) != (3,1).
adjacent(P1,P2) :- loc(P1), P1=(X,Y), loc(P2), P2=(X,Y+1).
adjacent(P1,P2) :- loc(P1), P1=(X,Y), loc(P2), P2=(X+1,Y).
adjacent(P1,P2) :- loc(P1), P1=(X,Y), loc(P2), P2=(X,Y-1).
adjacent(P1,P2) :- loc(P1), P1=(X,Y), loc(P2), P2=(X-1,Y).

robot(worker).
robot(charger).

1 { at(R,P,0) : loc(P) } 1 :- robot(R).

#program domain(__t).

0 { move(R,P,__t) : loc(P) } 1 :- robot(R).





#program domain_integrity(__t).

:- robot(R), { at(R,P,__t) : loc(P) } 0.
:- robot(R), 2 { at(R,P,__t) : loc(P) }.
:- loc(P), 2 { at(R,P,__t) : robot(R) }.



#program planning_problem(__t).
:- not at(worker,(1,1),0).
:- not at(charger,(2,2),0).
:- not at(worker

In [125]:
plans = tuple(evaluate_forwards(planning_problem_ps, parts=(
    ('base', ()),
    ('domain', ()),
    *(('domain', (clingo.Number(n),)) for n in range(4)),
    *(('domain_integrity', (clingo.Number(n),)) for n in range(5)),
    ('planning_problem', (clingo.Number(4),)),
    ),
    report=True))

Answer 1: {
loc((1,1))
loc((1,2))
loc((2,1))
loc((2,2))
loc((2,3))
loc((3,2))
loc((3,3))
robot(charger)
robot(worker)
adjacent((1,1),(1,2))
adjacent((1,1),(2,1))
adjacent((1,2),(1,1))
adjacent((1,2),(2,2))
adjacent((2,1),(1,1))
adjacent((2,1),(2,2))
adjacent((2,2),(1,2))
adjacent((2,2),(2,1))
adjacent((2,2),(2,3))
adjacent((2,2),(3,2))
adjacent((2,3),(2,2))
adjacent((2,3),(3,3))
adjacent((3,2),(2,2))
adjacent((3,2),(3,3))
adjacent((3,3),(2,3))
adjacent((3,3),(3,2))
at(charger,(2,2),0)
at(charger,(2,2),4)
at(charger,(3,2),1)
at(charger,(3,2),2)
at(charger,(3,2),3)
at(worker,(1,1),0)
at(worker,(2,1),1)
at(worker,(2,2),2)
at(worker,(2,3),3)
at(worker,(3,3),4)
move(charger,(2,2),3)
move(charger,(3,2),0)
move(worker,(2,1),0)
move(worker,(2,2),1)
move(worker,(2,3),2)
move(worker,(3,3),3)
}
Answer 2: {
loc((1,1))
loc((1,2))
loc((2,1))
loc((2,2))
loc((2,3))
loc((3,2))
loc((3,3))
robot(charger)
robot(worker)
adjacent((1,1),(1,2))
adjacent((1,1),(2,1))
adjacent((1,2),(1,1))
adjacent((1,2),(2,2))

In [ ]:
diagnosis_domain_str = """

#program diagnosis_domain.

{broken(R,P,0) : robot_component(R,P) }

#program diagnosis_domain(__t).

-broken(R,P,__t) :- not broken(R,P,__t).
broken(R,P,__t+1) :- not -broken(R,P,__t+1), -broken(R,P,__t).
broken(R,P,__t+1) :- broken(R,P,__t).

"""